In [1]:
from lxml import html

from bs4 import BeautifulSoup
import time
import datetime

import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os

from apscheduler.schedulers.background import BackgroundScheduler
import sys

In [2]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [3]:
class_schedules_folder_path = './class_schedules'

In [4]:
course_schedule = []

In [5]:
#setup option for chrome profile
chrome_options = Options()
chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
chrome_options.add_argument("--remote-debugging-port=9222") 
prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)


#start web driver
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# driver = webdriver.Chrome('/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver', options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)

driver.implicitly_wait(10)

wait = WebDriverWait(driver, 10)

In [6]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element("xpath",xpath)

In [7]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)

In [8]:
def sign_in():

    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')

    # username = driver.find_element("id",'cphContents_txtUsername')
    # password = driver.find_element("id",'cphContents_txtPassword')

#     username.send_keys("DN20092360")
#     password.send_keys("jj1216")
    username.send_keys("DN23035242")
    password.send_keys("NTW3BSFH")
    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()


def click_book_for_class():
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '預訂課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses=course_schedule):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses=course_schedule):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results
def search_courses_by_name(name_str, courses=course_schedule):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses=course_schedule):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results
def get_course_schedule_table_title():
    return driver.find_element("xpath","//div[@class='tbl-header']").get_attribute('innerText')

def click_yedn_btn():
    print("1")
    sync_get_element_by_xpath(driver, "//[contains(text(), '敦南瑜珈會館')]")
    print("2")


#             btns = driver.find_elements_by_class_name("//[contains(text(), 'Book for Class')]")
            
def click_next_week_btn():

    if check_exists_by_xpath("//*[@class='btnCircleRight']"):

        btns = driver.find_elements(By.CLASS_NAME, 'btnCircleRight')
    else:
        return False
    
    if btns is not None:
        
        if not len(btns) == 1:
            return False
        
        btns[0].click()
        
        return True
    else:
        return False

def click_previous_week_btn():
    if check_exists_by_xpath("//*[@class='btnCircleLeft']"):
        btns = driver.find_elements_by_class_name('btnCircleLeft')
    else:
        print("@2")
        return False
    
    if btns is not None:
        
        assert len(btns) == 1
        
        btns[0].click()
        return True
    else:
        return False

In [9]:
driver.get(base_url)

In [10]:
sign_in()

In [11]:
start = time.time()
click_book_for_class()

In [12]:
def scrape_a_week():
    course_schedule = []
    header_titles = []

    room_td_elms = driver.find_elements("xpath","//div[@class='studios']/table/tbody/tr[1]/td")

    for room_idx in range(len(room_td_elms)):
        room_select_btn = room_td_elms[room_idx].find_element("xpath","./span")
        room_label = room_select_btn.find_element("xpath","./label")
        room_name = room_label.get_attribute('innerText')

#         time.sleep(1)
        if room_name == 'DN 6 PT Room':
            continue
        
        print(room_name)
        room_select_btn.click()
        room_td_elms = driver.find_elements("xpath","//div[@class='studios']/table/tbody/tr[1]/td")

        th_elms = driver.find_elements("xpath","//div[@class='tbl-container']/table/tbody/tr[1]/th")
        for idx, th_elm in enumerate(th_elms):
            header_title = th_elm.get_attribute("innerText")
            header_titles.append(header_title)

        td_elms = driver.find_elements("xpath","//div[@class='tbl-container']/table/tbody/tr[2]/td")

        title_name = get_course_schedule_table_title()
        schedule_filename = os.path.join(class_schedules_folder_path, title_name + '.json')
#         print("schedule_filename={}".format(schedule_filename))
            
        for idx, td_elm in enumerate(td_elms):
            #skip useless Column data of 'TIME'
            if idx == 0:
                continue
            a_elms = td_elm.find_elements("xpath","./a")

            for a_elm in a_elms:
                course = {}
                course['room'] = room_td_elms[room_idx].get_attribute("innerText")
                course['date'] = header_titles[idx]
                course['href'] = a_elm.get_attribute('href')
                span_1 = a_elm.find_element("xpath","./span[1]")
                course['name'] = span_1.get_attribute('innerText')
                span_2 = a_elm.find_element("xpath","./span[2]")
                tmp_str = span_2.get_attribute('innerText')

                teacher_str = tmp_str.split('\n')[0]
                time_str = tmp_str.split('\n')[1]

                course['teacher'] = teacher_norm(teacher_str)
                course['time'] = time_str

                course_schedule.append(course)
            with open(schedule_filename, 'w', encoding='utf-8') as fp:
                json.dump(course_schedule, fp)
    return schedule_filename, course_schedule


In [13]:
os.makedirs(class_schedules_folder_path, exist_ok=True)

div = driver.find_element("xpath","//span[contains(text(), '敦南瑜珈會館')]/..")
div.click()

time.sleep(3)
schedule_filename = ""
course_schedule = []

# get string from website
title_name = get_course_schedule_table_title()
print("title_name= "+title_name)
[start_date, end_date] = title_name.split(' - ')
# get start date from True Yoga Website weird convention of showing date 
conv_year = end_date.split(' ')[2]

start_date = start_date.replace("月","")

conv_day = start_date.split(' ')[0]
conv_month = start_date.split(' ')[1]

conv_month = conv_month.replace("一","1")
conv_month = conv_month.replace("二","2")
conv_month = conv_month.replace("三","3")
conv_month = conv_month.replace("四","4")
conv_month = conv_month.replace("五","5")
conv_month = conv_month.replace("六","6")
conv_month = conv_month.replace("七","7")
conv_month = conv_month.replace("八","8")
conv_month = conv_month.replace("九","9")
conv_month = conv_month.replace("十一","11")
conv_month = conv_month.replace("十二","12")
conv_month = conv_month.replace("十","10")

start_date = datetime.date(int(conv_year),int(conv_month),int(conv_day))
min_time = datetime.datetime.min.time()
start_datetime = datetime.datetime.combine(start_date, min_time)

# get end date from True Yoga Website weird convention of showing date 
end_date = end_date.replace("月","")
conv_day = end_date.split(' ')[0]
conv_month = end_date.split(' ')[1]

conv_month = conv_month.replace("一","1")
conv_month = conv_month.replace("二","2")
conv_month = conv_month.replace("三","3")
conv_month = conv_month.replace("四","4")
conv_month = conv_month.replace("五","5")
conv_month = conv_month.replace("六","6")
conv_month = conv_month.replace("七","7")
conv_month = conv_month.replace("八","8")
conv_month = conv_month.replace("九","9")
conv_month = conv_month.replace("十一","11")
conv_month = conv_month.replace("十二","12")
conv_month = conv_month.replace("十","10")

end_date = datetime.date(int(conv_year),int(conv_month),int(conv_day))
min_time = datetime.datetime.min.time()
end_datetime = datetime.datetime.combine(end_date, min_time)
# get current date time
class_datetime = datetime.datetime.now() + datetime.timedelta(days=3)

print("start_datetime: {} , class_datetime: {} , end_datetime: {}".format(start_datetime,class_datetime,end_datetime))

# decide if clicking next is needed
if not start_datetime<=class_datetime<=end_datetime:
    click_next_week_btn()

scrape_a_week()

title_name= 08 一月 - 14 一月 2024
08 一
start_datetime: 2024-01-08 00:00:00 , class_datetime: 2024-01-16 17:57:57.406402 , end_datetime: 2024-01-14 00:00:00
敦南 4 教室
敦南 6 教室
敦南 7 教室
敦南 9 教室


('./class_schedules/15 一月 - 21 一月 2024.json',
 [{'room': '敦南 4 教室',
   'date': '15 1月 周一',
   'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1264958',
   'name': 'Hatha Yoga 哈達瑜珈',
   'teacher': 'Una',
   'time': '07:30 - 08:30 上午'},
  {'room': '敦南 4 教室',
   'date': '15 1月 周一',
   'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1264959',
   'name': 'Yoga Tone',
   'teacher': 'Una',
   'time': '09:15 - 10:15 上午'},
  {'room': '敦南 4 教室',
   'date': '15 1月 周一',
   'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1264960',
   'name': 'Gentle Yoga 和緩瑜珈',
   'teacher': 'Ellen',
   'time': '11:00 - 12:00 下午'},
  {'room': '敦南 4 教室',
   'date': '15 1月 周一',
   'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1264961',
   'name': 'Vinyasa Intro 流暢瑜珈入門',
   'teacher': 'Ethan Lee',
   'time': '02:30 - 03:45 下午'},
  {'room': '敦南 4 教室',
   'date': '15 1月 周一',
   'href'

In [14]:
print("Time elapsed: {:.2f}".format(time.time() - start))

Time elapsed: 7.97


In [15]:
driver.quit()